In [ ]:
import pandas as pd
import pickle5 as pickle
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, GlobalMaxPooling1D, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding,  Concatenate, Lambda
from keras.models import Model
from sklearn.metrics import roc_auc_score,roc_curve, auc
from numpy import random
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from tensorflow.keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
import seaborn as sns


import functions as f

In [ ]:
def load_data(D,randomize=False):
    try:
        with open("df_train_"+str(D)+"D.pkl", "rb") as fh:
          df_train = pickle.load(fh)
    except:
        df_train = pd.read_pickle("hyppi-train-data.pkl")
    try:
        with open("df_test_"+str(D)+"D.pkl", "rb") as fh:
          df_test = pickle.load(fh)
    except:
        df_test = pd.read_pickle("hyppi-test-data.pkl")
    if randomize:
        return f.shuff_together(df_train,df_test)
    else:
        return df_train,df_test

df_train,df_test = load_data(3)

In [ ]:
data_join,data_test_join,num_words_join,MAX_SEQUENCE_LENGTH_,MAX_VOCAB_SIZE_ = f.get_seq_data_join(5000,2000,df_train,df_test,pad='pre')

In [ ]:
DROP = 0.4
BATCH_SIZE = 64
EPOCHS = 10
ip = f.transf_model(MAX_SEQUENCE_LENGTH_,num_words_join,5,DROP)
x = Dropout(DROP)(ip.output)
x = Dense(128, activation="relu")(x)
x = Dropout(DROP)(x)
outputs = Dense(1, activation="sigmoid")(x)
model1D_CNN_join=Model(ip.input,outputs)

model1D_CNN_join.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.01), metrics=['accuracy'])
model1D_CNN_join.fit(data_join, df_train['label'].values,batch_size= BATCH_SIZE,epochs=EPOCHS, validation_data=(data_test_join,df_test['label'].values))
print(roc_auc_score(df_test['label'].values, model1D_CNN_join.predict(data_test_join)))